In [1]:
import torch
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
from transformers import BertTokenizer
from torch.utils.data import TensorDataset
from transformers import BertForSequenceClassification
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from transformers import AdamW, get_linear_schedule_with_warmup
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split

In [2]:
batch_size = 8
epochs = 10

In [24]:
import random
seed_val = 1
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted')

def accuracy_per_class(preds, labels):
    label_dict_inverse = {v: k for k, v in polarity_dict.items()}
    
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        print(f'Class: {label_dict_inverse[label]}')
        print(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)}\n')
        
def evaluate(dataloader_val):

    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in dataloader_val:
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals

def evaluate_single(sentense):

    model.eval()
    
    loss_val_total = 0
    encoded_data_test = tokenizer.batch_encode_plus(
    [sentense], 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=256, 
    return_tensors='pt'
    )
    encoded_data_test.to(device)
    with torch.no_grad():        
        outputs = model(**encoded_data_test)
    return outputs

In [5]:
df = pd.read_csv('data/polarity3_data/twitter_polarity3.csv', encoding='latin-1')
#df.set_index('id', inplace=True)
df.head()

,text,sentiment
0,@stellargirl I loooooooovvvvvveee my Kindle2. ...,positive
1,Reading my kindle2... Love it... Lee childs i...,positive
2,"Ok, first assesment of the #kindle2 ...it fuck...",positive
3,@kenburbary You'll love your Kindle2. I've had...,positive
4,@mikefish Fair enough. But i have the Kindle2...,positive


In [6]:
#df=df.sample(n=10000,axis=0)
df.shape

(498, 2)

In [14]:
polarity_dict={'positive':0, 'negative':1, 'neutral':2}
df['label'] = df.sentiment.replace(polarity_dict)

In [15]:
df

,text,sentiment,label
0,@stellargirl I loooooooovvvvvveee my Kindle2. ...,positive,0
1,Reading my kindle2... Love it... Lee childs i...,positive,0
2,"Ok, first assesment of the #kindle2 ...it fuck...",positive,0
3,@kenburbary You'll love your Kindle2. I've had...,positive,0
4,@mikefish Fair enough. But i have the Kindle2...,positive,0
...,...,...,...
493,Ask Programming: LaTeX or InDesign?: submitted...,neutral,2
494,"On that note, I hate Word. I hate Pages. I hat...",negative,1
495,Ahhh... back in a *real* text editing environm...,positive,0
496,"Trouble in Iran, I see. Hmm. Iran. Iran so far...",negative,1


In [16]:
X_train, X_val, y_train, y_val = train_test_split(df.text.values, 
                                                  df.label.values, 
                                                  test_size=0.10, 
                                                  random_state=1, 
                                                  stratify=df.label.values)


In [17]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

encoded_data_train = tokenizer.batch_encode_plus(
    X_train, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=300, 
    return_tensors='pt'
)

encoded_data_val = tokenizer.batch_encode_plus(
    X_val,
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=300, 
    return_tensors='pt'
)

In [18]:
input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(y_train)

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(y_val)

dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)

In [20]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=len(polarity_dict),
                                                      output_attentions=True,
                                                      output_hidden_states=False)


dataloader_train = DataLoader(dataset_train, 
                              sampler=RandomSampler(dataset_train), 
                              batch_size=batch_size)

dataloader_validation = DataLoader(dataset_val, 
                                   sampler=SequentialSampler(dataset_val), 
                                   batch_size=batch_size)

In [21]:

optimizer = AdamW(model.parameters(),
                  lr=1e-5, 
                  eps=1e-8)
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps=0,
                                            num_training_steps=len(dataloader_train)*epochs)
model.to(device)

pass

In [22]:
for epoch in tqdm(range(1, epochs+1)):
    
    model.train()
    
    loss_train_total = 0

    progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    for batch in progress_bar:

        model.zero_grad()
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }       

        outputs = model(**inputs)
        
        loss = outputs[0]
        loss_train_total += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()
        
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
         
        
    torch.save(model.state_dict(), f'finetuned_BERT_epoch_{epoch}.model')
        
    tqdm.write(f'\nEpoch {epoch}')
    
    loss_train_avg = loss_train_total/len(dataloader_train)            
    tqdm.write(f'Training loss: {loss_train_avg}')
    
    val_loss, predictions, true_vals = evaluate(dataloader_validation)
    val_f1 = f1_score_func(predictions, true_vals)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score (Weighted): {val_f1}')


Epoch 1
Training loss: 1.0522279632943017
Validation loss: 0.9813379645347595
F1 Score (Weighted): 0.5386364692218351



Epoch 2
Training loss: 0.8874638729861805
Validation loss: 0.851930056299482
F1 Score (Weighted): 0.635708245243129



Epoch 3
Training loss: 0.7380711627858025
Validation loss: 0.7244529213224139
F1 Score (Weighted): 0.779778208061961



Epoch 4
Training loss: 0.5810519164162022
Validation loss: 0.6284599431923458
F1 Score (Weighted): 0.8214286563968358



Epoch 5
Training loss: 0.4695601508553539
Validation loss: 0.5873121406350817
F1 Score (Weighted): 0.7840430107526882



Epoch 6
Training loss: 0.3622368124446699
Validation loss: 0.5161647072860173
F1 Score (Weighted): 0.8401836441893832



Epoch 7
Training loss: 0.2889295473162617
Validation loss: 0.5093615991728646
F1 Score (Weighted): 0.8234390415785765



Epoch 8
Training loss: 0.24731446935662202
Validation loss: 0.4786973255021231
F1 Score (Weighted): 0.8411798223562929



Epoch 9
Training loss: 0.21980558255953447
Validation loss: 0.47070581572396414
F1 Score (Weighted): 0.8193406593406592



Epoch 10
Training loss: 0.19961506021874292
Validation loss: 0.46751950894083294
F1 Score (Weighted): 0.8216026936026936



In [25]:
_, predictions, true_vals = evaluate(dataloader_validation)
accuracy_per_class(predictions, true_vals)

Class: positive
Accuracy: 16/18

Class: negative
Accuracy: 14/18

Class: neutral
Accuracy: 11/14



In [ ]:
import gc
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
def reset_gpu():
    try:
        del encoded_data_test
    except:
        pass
    try:
        del tokenizer
    except:
        pass
    try:
        del output
    except:
        pass
    try:
        del scores, dic
    except:
        pass
    torch.cuda.empty_cache()
print(torch.cuda.memory_allocated(), torch.cuda.memory_reserved())
gc.collect()
reset_gpu()
print(torch.cuda.memory_allocated(), torch.cuda.memory_reserved())

In [ ]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=3,
                                                      output_attentions=True,
                                                      output_hidden_states=True)
model.to(device)
model.load_state_dict(torch.load('finetuned_BERT_epoch_10.model', map_location=torch.device('cuda')))

In [ ]:
print(torch.cuda.memory_allocated(), torch.cuda.memory_reserved())
reset_gpu()
print(torch.cuda.memory_allocated(), torch.cuda.memory_reserved())

inputs=['Imagine all the people sharing all the world',
        'I love this restaurant but the line is too long',
        'we should be making a profit by this time next year.',
        'She earned her PhD in physics before becoming a postdoctoral fellow at Rockefeller University, where she worked on developing and implementing an underwater touchscreen for dolphins.',
        'I did smile today!',
        'Applies the Softmax function to an n-dimensional input Tensor', 
        'I have not slept enough today', 
        'the weather is crazy hot today',
        'Listen Morty, I hate to break it to you, but what people call “love” is just a chemical reaction that compels animals to breed. It hits hard, Morty, then it slowly fades, leaving you stranded in a failing marriage. I did it. Your parents are gonna do it. Break the cycle, Morty. Rise above. Focus on science.',
        'I am a scientist, because I invent, transform, create, and destroy for a living, and when I do not like something about the world, I change it',
        'I turned myself into a pickle, Morty! I’m Pickle Rick!',
        'Do you wanna develop an app?',
        'Honey, stop raising your father cholesterol so you can take a hot funeral selfie.',
        'To live is to risk it all; otherwise you are just an inert chunk of randomly assembled molecules drifting wherever the universe blows you...'
        ]
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', 
                                          do_lower_case=True)
encoded_data_test = tokenizer.batch_encode_plus(
    inputs, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=256, 
    return_tensors='pt'
)
encoded_data_test.to(device)
output=model(**encoded_data_test)


In [ ]:
scores=torch.max(torch.nn.Softmax(dim=1)(output[0]),axis=1)
dic={0:'positive',1:'negative',2:'neutral'}
[(dic[int(scores[1][i])],round(float(x),3), inputs[i][:50]+ ('...' if len(inputs[i])>50 else '')) for i,x in enumerate(scores[0])]

In [ ]:
encoding = tokenizer.encode_plus(
  inputs[-2],
  max_length=100,
  add_special_tokens=True, # Add '[CLS]' and '[SEP]'
  return_token_type_ids=True,
  pad_to_max_length=True,
  return_attention_mask=True,
  return_tensors='pt',  # Return PyTorch tensors
)
tokenizer.convert_ids_to_tokens(encoding['input_ids'][0])

In [ ]:
encoding.keys()

In [ ]:
df=pd.read_json('Data/test.json',orient='split')
df=pd.concat([jdf,pd.DataFrame(jdf['sentence'].to_list())],axis=1).drop('sentence',axis=1)